In [1]:
from glob import glob
import simplejson as sjson
import gensim
from gensim.parsing.preprocessing import STOPWORDS
from gensim.utils import simple_preprocess 
from gensim import corpora
from gensim.corpora import Dictionary  

In [2]:
filepath="../data/fyear2/*"

In [3]:
files = sorted(glob(filepath))
data = []
for file in files:
    for line in open(file):
        tmp=sjson.loads(line)
        data.append((tmp['gvkey'],tmp['comp_name'],tmp['item_1']))

In [4]:
len(data)

105765

In [5]:
data = pd.DataFrame(data)
data.columns = ['gvkey','comp_name','item1']

data.head()

,gvkey,comp_name,item1
0,5589,HERCULES INC,"ITEM 1. BUSINESS: Hercules Incorporated ""Hercu..."
1,5600,HERTZ CORP,ITEM 1. BUSINESS. GENERAL The Hertz Corporatio...
2,5142,GERBER SCIENTIFIC INC,Item 11. Executive Compensation . . . . . . . ...
3,5016,INVESTORS INSURANCE GROUP INC,Item 1. Description of Business Business Devel...
4,5018,GENCORP INC,ITEM 1. BUSINESS GenCorp Inc. hereinafter the ...


In [6]:
documents = data['item1'].tolist()

In [7]:
def tokenize(text):  
    return [token for token in simple_preprocess(text.lower()) if token not in STOPWORDS]  

In [ ]:
%%time 

test = data['item1'].apply(lambda x: tokenize(x))

In [ ]:
processed_docs = [tokenize(doc) for doc in documents]  #tokenize

In [ ]:
import pickle

with open("test.pickle", 'wb') as f:
    pickle.dump(processed_docs,f)
    
with open("test.pickle", 'rb') as f:
    processed_docs = pickle.load(f)


In [44]:
dictionary = Dictionary(processed_docs)
#dictionary.save('../model/10kdict.dict')  # store the dictionary, for future reference

In [45]:
dictionary.save('../model/10kdict.dict')

In [9]:
dictionary = corpora.Dictionary.load('../model/10kdict.dict')

In [10]:
print(dictionary)

Dictionary(357889 unique tokens: ['item', 'business', 'hercules', 'incorporated', 'company']...)


In [10]:
dictionary.filter_extremes(no_below=20, no_above=0.1)
dictionary.compactify()

In [11]:
print(dictionary)

Dictionary(63593 unique tokens: ['hercules', 'divested', 'composite', 'graphite', 'aerospace']...)


In [12]:
dictionary.save('../model/10kdict1.dict')

In [13]:
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [14]:
corpora.MmCorpus.serialize('../model/10kcorpus.mm', corpus)